# Thermoelasticity-based modal damage identification

## Import packages

In [5]:
import numpy as np
import matplotlib.pyplot as plt
import pysfmov as sfmov
from ThermalData import *

## Input parameters

### Thermal video

In [6]:
k = 6.51 # slope endurance curve
B = 800.26 # endurance curve 
C = B**k

In [7]:
filename = './data/rec.sfmov'
fs = 400
dt = 1 / fs

data = sfmov.get_data(filename)
km = 1.2 * 10**(-8) 
stress = 10 * (data / km ) * 10**-6 # stress in MPa

location = 39,79,3,3 #vert mode area 
f = 55 # [54, 56] # HZ


In [8]:
td = ThermalData(stress, dt)

In [10]:
%matplotlib qt
x , _ = td.nf_identification()

NameError: name 'dt' is not defined

In [ ]:
m = td.get_life(C, k, 'Modal', f = f, location = location, f_span = 0.1)
tb = td.get_life(C, k, 'TovoBenasciutti', location = location)
d = td.get_life(C, k, 'Dirlik', location = location)
r = td.get_life(C, k, 'Rainflow', location = location)

In [ ]:
print(f'          Rainflow: {r:4.0f} s')
print(f'            Dirlik: {d:4.0f} s')
print(f'  Tovo-Benasciutti: {tb:4.0f} s')
print(f'             Modal: {m:4.0f} s')

In [ ]:
tb = td.get_life(C, k, 'TovoBenasciutti')
m = td.get_life(C, k, 'Modal', f = f)

In [ ]:
plt.figure()
plt.imshow(m)
plt.colorbar()
plt.clim(1e10,1e19)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt 
from tqdm import tqdm
import FLife



class ThermalData():
        
    def __init__(self, x, dt):
        
        self.x = x
        self.dt = dt

        self.N = self.x.shape[0]
        self.self.ds = self.x - self.x[0,:,:]
    
    def _find_nearest(self, array, value):
        
        array = np.asarray(array)
        idx = (np.abs(array - value)).argmin()
        
        return array[idx], idx
    
    def _pixel_selection(self, event):
        
        global ix, iy
        ix, iy = event.xdata, event.ydata
        plt.close()
    
    def nf_identification(self, location = None, high_pass = 5, low_pass = 100) :
                
        if location is None:
            global ix, iy
            fig = plt.figure()
            ax = fig.add_subplot(111)
            ax.imshow(self.x[0,:,:])
            fig.canvas.mpl_connect('button_press_event', self._pixel_selection)
            plt.show()
            (x, y, w, h) = (int(ix) - 2, int(iy) - 2, 5, 5)
        else:
            (x, y, w, h) = location    
        
        self.ds = self.ds[:, y:(y+h), x:(x+w)]
                
        fft = np.abs(np.fft.rfft(self.ds, self.N, axis = 0) * 2 / self.N)
        freq = np.fft.rfftfreq(self.N, self.dt)
        
        y_peak = np.max(fft[(freq > high_pass) & (freq < low_pass)])
        x_peak = freq[np.where(fft == y_peak)[0][0]]
                        
        return x_peak, y_peak
    
    def get_life(self, C, k, method = None, f = None, location = None, f_span = None):
        
        if method == None:
            method = 'TovoBenasciutti'
            
        if f_span == 0:
            raise ValueError('Frequency span around natural frequency must not be zero: set it to "None" or to a float value.')
            
        if method not in ['Modal', 'TovoBenasciutti', 'Dirlik', 'Rainflow']:
            raise ValueError('Method must be one of: Modal, TovoBenasciutti, Dirlik, Rainflow')
        
        if location is not None:
            (x, y, w, h) = location    
            self.ds = self.ds[:, y:(y+h), x:(x+w)]
        
        life = np.zeros(shape=(self.ds.shape[1], self.ds.shape[2]))
        npixels = self.ds.shape[1] * self.ds.shape[2]  
        
        if method == 'Modal':
            
            if f == None:
                raise ValueError('Natural frequency must be defined')
                
            with tqdm(total = npixels) as pbar:   
                for i in range(self.ds.shape[1]):
                    for j in range(self.ds.shape[2]):    
                        
                        fft  = np.abs(np.fft.rfft(self.ds[:,i,j], self.N) * 2 / self.N)
                        freq = np.fft.rfftfreq(self.N, self.dt)
            
                        if f_span is not None:
                            x_peak = freq[(freq >= f - f_span) & (freq <= f + f_span)]
                            y_peak = fft[(freq >= f - f_span) & (freq <= f + f_span)]
                            damage = np.sum(x_peak / (C / y_peak**k))
                            
                        else:
                            x_peak = self._find_nearest(freq, f)[0]    
                            y_peak = fft[self._find_nearest(freq, f)[1]]    
                            damage = x_peak / (C / y_peak**k)
            
                        life[i,j] = 1 / damage
                        pbar.update(1)
        
        elif method == 'TovoBenasciutti':
            
            with tqdm(total = npixels) as pbar:
                for i in range(self.ds.shape[1]):
                    for j in range(self.ds.shape[2]):
                        
                        tb = FLife.TovoBenasciutti(FLife.SpectralData(self.ds[:,i,j], self.dt))
                        life[i,j] = tb.get_life(C = C, k = k, method = "method 2")
                        pbar.update(1)
                    
        elif method == 'Dirlik':
            
            with tqdm(total = npixels) as pbar:
                for i in range(self.ds.shape[1]):
                    for j in range(self.ds.shape[2]):
                        
                        dirlik = FLife.Dirlik(FLife.SpectralData(self.ds[:,i,j], self.dt))
                        life[i,j] = dirlik.get_life(C = C, k = k)
                        pbar.update(1)
                    
        elif method == 'Rainflow':
            
            with tqdm(total = npixels) as pbar:    
                for i in range(self.ds.shape[1]):
                    for j in range(self.ds.shape[2]):
                        
                        rf = FLife.Rainflow(FLife.SpectralData(self.ds[:,i,j], self.dt))
                        life[i,j] = rf.get_life(C = C, k = k)
                        pbar.update(1)

        if location is not None:
            return np.mean(life, axis = (0,1))
        else:
            return life